<a href="https://colab.research.google.com/github/pk2971/computational-gender-analysis/blob/main/notebooks/LLM_summary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install -U langchain-community


In [ ]:
%pip install -U sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 470.2/470.2 kB 10.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 121.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 94.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 42.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 112.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Unins

In [ ]:
%pip install chromadb


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.5/19.5 MB 108.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 82.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 111.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.7/55.7 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.2/196.2 kB 19.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 10.2 MB/s 

In [ ]:
%pip install gradio


In [ ]:
import zipfile
import re
from typing import Union, List
from bs4 import BeautifulSoup
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from sentence_transformers.cross_encoder import CrossEncoder
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from langchain.vectorstores import Chroma
import gradio as gr

In [ ]:
def extract_text_from_xml(xml_content: str) -> str:
    soup = BeautifulSoup(xml_content, "lxml-xml")
    return soup.get_text(separator=" ", strip=True)

def load_and_split_xml_from_zip(
    zip_path: str,
    years: Union[int, List[int]],
    chunk_size: int = 1000,
    chunk_overlap: int = 200
):
    if isinstance(years, int):
        years = [years]
    year_pattern = '|'.join(str(y) for y in years)

    with zipfile.ZipFile(zip_path, "r") as zip_file:
        matched_files = [
            f for f in zip_file.namelist()
            if re.search(rf'debates({year_pattern})-\d{{2}}-\d{{2}}a\.xml', f)
        ]

        all_texts = []
        for filename in matched_files:
            with zip_file.open(filename) as file:
                xml_content = file.read().decode("utf-8", errors="ignore")
                text = extract_text_from_xml(xml_content)
                all_texts.append(text)

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    docs = text_splitter.create_documents(all_texts)
    return docs


In [ ]:
# Change year of documents to load here.
zip_file_path = "/content/drive/MyDrive/debates.zip"
years = 1928
docs = load_and_split_xml_from_zip(zip_file_path, years)


In [ ]:
embedding_model = "BAAI/bge-small-en-v1.5"
embeddings = HuggingFaceEmbeddings(model_name=embedding_model)


/tmp/ipython-input-8-334468853.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=embedding_model)
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.war

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/366 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
vectorstore = Chroma.from_documents(docs, embeddings)


In [ ]:
reranker = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')


config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

In [ ]:
model_name = "mistralai/Mistral-7B-Instruct-v0.2"
access_token = "HF token"

tokenizer = AutoTokenizer.from_pretrained(model_name, token=access_token)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    token=access_token,
    device_map="auto"
)


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [ ]:
# Returns the answer with context from the Debates for cross referencing
def gradio_rag_bot_context(query):
    # Retrieve and rerank
    retrieved_docs = vectorstore.similarity_search(query, k=15)
    texts = [doc.page_content for doc in retrieved_docs]
    pairs = [[query, t] for t in texts]
    scores = reranker.predict(pairs)
    ranked = sorted(zip(texts, scores), key=lambda x: x[1], reverse=True)
    top_contexts = [t for t, s in ranked[:5]]
    context = "\n".join(top_contexts)
    # Improved prompt
    prompt = (
    "You are a historian assistant. Based on the context below, answer the question.\n"
    "Context:\n"
    f"{context}\n\n"
    f"Question: {query}\n"
    "Answer (as bullet points):"
)

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    output = model.generate(**inputs, max_new_tokens=256)
    answer = tokenizer.decode(output[0], skip_special_tokens=True)
    # Show context for debugging
    return f"ANSWER:\n{answer}"

iface = gr.Interface(
    fn=gradio_rag_bot_context,
    inputs=gr.Textbox(lines=2, label="Ask a question about the debates"),
    outputs=gr.Textbox(lines=20, label="Debug Output (Context + Answer)"),
    title="Debates RAG Chatbot (Debug Mode)",
    description="See the context the model is using. If the answer is off, the context is likely not relevant."
)


In [ ]:
iface.launch(share=True)  # 'share=True' gives you a public link in Colab


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://a253fce8c861b11493.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
def gradio_rag_bot(query):
    # Retrieve and rerank
    retrieved_docs = vectorstore.similarity_search(query, k=15)
    texts = [doc.page_content for doc in retrieved_docs]
    pairs = [[query, t] for t in texts]
    scores = reranker.predict(pairs)
    ranked = sorted(zip(texts, scores), key=lambda x: x[1], reverse=True)
    top_contexts = [t for t, s in ranked[:5]]
    context = "\n".join(top_contexts)

    # Build improved prompt
    prompt = (
        "You are a historian assistant. Based on the context below, answer the question.\n"
        "Context:\n"
        f"{context}\n\n"
        f"Question: {query}\n"
        "Answer (as bullet points):"
    )

    # Generate answer
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    output = model.generate(**inputs, max_new_tokens=512)
    answer = tokenizer.decode(output[0], skip_special_tokens=True)

    # Return only the clean answer

    if "Answer (as bullet points):" in answer:
      answer = answer.split("Answer (as bullet points):")[-1].strip()

    return answer


In [ ]:
iface = gr.Interface(
    fn=gradio_rag_bot,
    inputs=gr.Textbox(lines=2, label="Ask a question about the debates"),
    outputs=gr.Textbox(lines=20, label="Answer"),
    title="Debates RAG Chatbot",
    description="Ask about historical UK Parliament debates (1919–2024)."
)


In [ ]:
iface.launch(share=True)  # 'share=True' gives you a public link in Colab


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://827e650e1c5493c5fd.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
